# Interpolation and extrapolation

Every Lagrangian cell comes with the necessary tools to interpolate or extrapolate multidimensional data. This can be done manually using the shape functions and shape function matrices of the cells, but there is an easier and quicker way that uses these in the background for us. Take the 8-noded hexahedron as an example.

In [6]:
from sigmaepsilon.mesh.cells import H8
master_coordinates = H8.master_coordinates()

Every cell is equipped with an interpolator.

In [7]:
help(H8.interpolator)

Help on method interpolator in module sigmaepsilon.mesh.cells.base.cell:

interpolator(x: Iterable = None) -> Callable method of sigmaepsilon.mesh.abcdata.ABCMeta_MeshData instance
    Returns a callable object that can be used to interpolate over
    nodal values of one or more cells.
    
    Parameters
    ----------
    x: Iterable, Optional
        The locations of known data. If provided here, some calculations
        can be cached and reusing the interpolatior becomes cheaper.
        
    Returns
    -------
    Callable
        A function that either takes 2 or 3 arguments. It the source coordinates
        were provided when calling this function, the returned interpolator function
        only takes 2 arguments.
        
    Examples
    --------
    Let assume that we know some data at some locations:
    >>> source_data = [1, 2, 3, 4]
    >>> source_location = [[-0.5, -0.5], [0.5, -0.5], [0.5, 0.5], [-0.5, 0.5]]
    
    We want to extrapolate from this information to the

This is a factory function that returns another callable, that we use for interpolation or extrapolation of known data at known locations. Note that the mechanism uses only class level information, therefore there is no need to create an instance.

In [8]:
interpolator = H8.interpolator()
help(interpolator)

Help on function interpolator in module sigmaepsilon.mesh.cells.base.cell:

interpolator(x_source: Iterable, values_source: Iterable, x_target: Iterable) -> Union[float, numpy.ndarray]
    Returns interpolated values from a set of known points and values.



We have to feed the interpolator with the locations of the known data, the knowd data itself and the locations where we want to know the data:

In [9]:
source_coordinates = master_coordinates / 2
source_values = [1, 2, 3, 4, 5, 6, 7, 8]
target_coordinates = master_coordinates

interpolator(source_coordinates, source_values, target_coordinates)

array([-3.5,  0.5,  0.5,  4.5,  4.5,  8.5,  8.5, 12.5])

It is possible to pass the source coordinates to the factory function. This is useful if we plan to reuse the interpolator with the same source points and can save a little time. In this case only the source values and the target points need to be provided.

In [10]:
source_coordinates = master_coordinates / 2
source_values = [1, 2, 3, 4, 5, 6, 7, 8]
target_coordinates = master_coordinates

interpolator = H8.interpolator(source_coordinates)
interpolator(source_values, target_coordinates)

array([-3.5,  0.5,  0.5,  4.5,  4.5,  8.5,  8.5, 12.5])

As noted in the documentation, if the number of source coorindates does not match the number of nodes (and hence the number of shape functions) of the master element of the class, the interpolation is gonna be under or overdetermined and the operation involves the calculation of a generalized inverse. This means, that for instance feeding a 4-noded quadrilateral with 9 source points and data values is more information than what the class is normally able to precisely handle and the resulting interpolator will represent a fitting function. In that case, if you want a precise interpolation, you would want to use a 9-node quadrilateral, or accept the loss of information.

In [11]:
from sigmaepsilon.mesh.cells import Q4, Q9

master_coordinates = Q9.master_coordinates()
source_coordinates = master_coordinates / 2
source_values = [i+1 for i in range(9)]
target_coordinates = master_coordinates

interpolator = Q4.interpolator()
interpolator(source_coordinates, source_values, target_coordinates)

array([1.66666667, 4.33333333, 4.33333333, 9.66666667, 3.        ,
       4.33333333, 7.        , 5.66666667, 5.        ])

Usiing the 9-noded quadrilateral is a better choice here and you can have an exact interpolation or extrapolation.

In [12]:
interpolator = Q9.interpolator()
interpolator(source_coordinates, source_values, target_coordinates)

array([-45., -29., -29., -37.,  -5.,  -1.,  -1.,   3.,   9.])

All is the same for one dimensional cells:

In [13]:
from sigmaepsilon.mesh.cells import L3

master_coordinates = L3.master_coordinates()
source_coordinates = master_coordinates / 2
source_values = [i+1 for i in range(3)]
target_coordinates = master_coordinates

interpolator = L3.interpolator()
interpolator(source_coordinates, source_values, target_coordinates)

array([0., 2., 4.])